## Random Forest

Notebook with implementation of the Random Forest algorithm to predict victory in Dota 2

-------------------------------------------------------------------------------------------------------------------------------

#### If you are running this code on Google Colab, you need to first upload the following feature file: *dota2_regular_features.csv*

## Regular matches

Useful functions to use to explore the data and preprocessing steps before feeding the data into the algorithm:

* df.columns : to see the names of the columns (i.e., features)
* df.dtype : to see the types in the data
* data.head()
* data.info()
* df.describe()

In [ ]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score, roc_curve, auc
import statistics as st

In [ ]:
# NOTE: uncomment this cell if you are running this code on a local machine. Please adjust the following variables to correctly point to the feature file location on your machine

# # Set directory for the regular match group
# cwd = os.getcwd()
# root_directory = os.path.dirname(cwd)

# regular_data_dir = root_directory + "\\model_features_pre-match\\regular\\"
# path_to_features = regular_data_dir + "dota2_regular_features.csv"

In [ ]:
# NOTE: use this cell if you are running this code on Google Colab

# Set directory for the regular match group. Make sure the feature file is uploaded to this Colab session
path_to_features = "/content/dota2_regular_features.csv"

In [ ]:
# Read the data (model feature file)
feature_regular_df = pd.read_csv(path_to_features)

### Exploration and preprocessing of the data

In [ ]:
# Print feature names
feature_regular_df.columns

In [ ]:
# Drop first column (match id)
feature_regular_df = feature_regular_df.drop(['match_id'], axis=1)

In [ ]:
# Existing types
feature_regular_df.dtypes

In [ ]:
feature_regular_df.head()

In [ ]:
feature_regular_df.info()

In [ ]:
# Fill in missing values with the median value of the feature
feature_regular_df = feature_regular_df.fillna(feature_regular_df.median())

In [ ]:
feature_regular_df['rad_first_pick'] = feature_regular_df['rad_first_pick'].astype(int)

### Model building, training and evaluation

In [ ]:
# Import random forest library
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Split into features (X) and label (y)
X, y = feature_regular_df.iloc[:,:-1],feature_regular_df.iloc[:,-1]

### Grid search

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [ ]:
# Create the grid of parameters
param_grid = {
    'bootstrap': [True],
    'max_depth': [5, 10, 15, 50],
    'max_features': ['auto', 'sqrt', 'log2'],
    'n_estimators': [50, 100, 200, 300]
}

# Create a based model
rf = RandomForestClassifier()

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = 2, verbose = 2)

In [ ]:
# Perform search and print the best parameters
grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 3 folds for each of 48 candidates, totalling 144 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  1.6min
[Parallel(n_jobs=2)]: Done 144 out of 144 | elapsed: 16.4min finished


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 200}

Best parameters:
{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'n_estimators': 200}

In [ ]:
# Create the model using the best paramters
model = RandomForestClassifier(bootstrap = True,
                               n_estimators=200, 
                               max_depth=10,
                               max_features='auto')

In [ ]:
features = [c for c in feature_regular_df.columns if c != 'win_label']
target = 'win_label'

In [ ]:
# Define the number of folders for the k-fold cross-validation
kfolds = KFold(n_splits=10, shuffle=True)

In [ ]:
# NOTE: the training process might take a while to execute

auc = list()

for train_idx, test_idx in kfolds.split(X):
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_test, y_test = X.iloc[test_idx], y.iloc[test_idx]  
    
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    
    auc.append(roc_auc_score(y_test, preds))

'Median AUC: {:.04f}'.format(st.median(auc))

'Median AUC: 0.5950'